In [6]:
from cgi import test
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#tf.keras.preprocessing.image.ImageDataGenerator
dir = "/content/Dataset_Folder/real_and_fake_face"
train_data = os.path.join(dir,'training_real')  # <base_dir,'dir to be conside'>
test_data = os.path.join(dir,'testing_fake')

img_h,img_w =150,150
batch_size = 32

#processing of these images

train_dat = ImageDataGenerator(
    rescale = 1./255,
    width_shift_range=0.2,
    height_shift_range=0.2,
    rotation_range=40,
    zoom_range=0.2,
)

test_dat = ImageDataGenerator(
    rescale = 1./255
)

train_gen = train_dat.flow_from_directory(
    train_data,
    target_size = (img_h,img_w),
    batch_size = batch_size,
    class_mode = 'binary'
)

test_gen = test_dat.flow_from_directory(
    test_data,
    target_size = (img_h,img_w),
    batch_size = batch_size,
    class_mode = 'binary'
)

#building a nn
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32,(3,3),activation = 'relu',input_shape = (img_h,img_w,3)),
    tf.keras.layers.MaxPool2D((2,2)),
    tf.keras.layers.Conv2D(32,(3,3),activation = 'relu',input_shape = (img_h,img_w,3)),
    tf.keras.layers.MaxPool2D((2,2)),
    tf.keras.layers.Conv2D(32,(3,3),activation = 'relu',input_shape = (img_h,img_w,3)),
    tf.keras.layers.MaxPool2D((2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512,activation = 'relu'),
    tf.keras.layers.Dense(1,activation = 'sigmoid')
])

#compile the output

model.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)

#training the model

tr=model.fit(train_gen,
               steps_per_epoch = 10,
               epochs = 10,
               validation_data = test_gen,
               validation_steps = 10
               )


FileNotFoundError: [WinError 3] The system cannot find the path specified: '/content/Dataset_Folder/real_and_fake_face\\training_real'

In [5]:
model.save('keras_model.h5')

In [2]:
from tensorflow.keras.models import load_model
model = load_model('keras_model.h5')

In [6]:
from keras.models import load_model  # TensorFlow is required for Keras to work
from PIL import Image, ImageOps  # Install pillow instead of PIL
import numpy as np

# Disable scientific notation for clarity
np.set_printoptions(suppress=True)

# Load the model
model = load_model("keras_Model.h5", compile=False)

# Load the labels
class_names = open("labels.txt", "r").readlines()

# Create the array of the right shape to feed into the keras model
# The 'length' or number of images you can put into the array is
# determined by the first position in the shape tuple, in this case 1
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

image = Image.open("Dataset_Folder/real_and_fake_face/training_fake/easy_152_0011.jpg")

# Resizing the image to be at least 224x224 and then cropping from the center
size = (224, 224)
image = ImageOps.fit(image, size, Image.Resampling.LANCZOS)

# Turn the image into a numpy array
image_array = np.asarray(image)

# Normalize the image
normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1

# Load the image into the array
data[0] = normalized_image_array

# Predicts the model
prediction = model.predict(data)
index = np.argmax(prediction)
class_name = class_names[index]
accuracy = prediction[0][index]

# Print Prediction and Accuracy
print("Class :", class_name[2:], end="")
print("Accuracy :", accuracy*100)


1/1 [==============================] - 1s 1s/step
Class : Fake
Accuracy : 99.99982118606567
